In [1]:
# Import libraries
import numpy as np

import pandas as pd
import os
import datetime
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot

import platform
from matplotlib import font_manager, rc


import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [52]:
# Preparate GRU
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from keras.layers import GRU
from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트

In [24]:
# Data preprocessing
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size+1):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size-1]))
    return np.array(feature_list), np.array(label_list)

def make_FCset(data, window_size):
    feature_list = []
    for i in range(len(data) - window_size+1):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
    return np.array(feature_list)

In [26]:
# Reading the input data
data_ori= pd.read_csv('total_data.csv')

In [29]:
data_ori.index=range(len(data_ori))

In [30]:
data_ori.columns

Index(['level_0', 'Time', 'Year', 'Week', 'Month', 'Rate', 'Mean temperature',
       'Minimum temperature', 'Maximum temperature', 'Rainfall',
       'Minimum huminity', 'Relative huminity', 'Day length',
       'Duration of sunshine', 'Soil temperature at 3m',
       'Soil temperature at 5m', 'Number of Daycare center',
       'Population of Daycare center', 'outbreak', 'warning', 'peak',
       'warning3', 'warning2', 'season', 'last_rate', 'th'],
      dtype='object')

In [33]:
targets=['warning']

In [34]:
data_ori.columns

Index(['level_0', 'Time', 'Year', 'Week', 'Month', 'Rate', 'Mean temperature',
       'Minimum temperature', 'Maximum temperature', 'Rainfall',
       'Minimum huminity', 'Relative huminity', 'Day length',
       'Duration of sunshine', 'Soil temperature at 3m',
       'Soil temperature at 5m', 'Number of Daycare center',
       'Population of Daycare center', 'outbreak', 'warning', 'peak',
       'warning3', 'warning2', 'season', 'last_rate', 'th'],
      dtype='object')

In [35]:
variables=['Minimum temperature',
       'Rainfall', 'Relative humidity', 'Day length',
       'Soil temperature at 3m',
       'Soil temperature at 5m', 'last_rate']

In [37]:
# model input
u_data=data_ori
scaler=MinMaxScaler()
window_size=3
layer_num=50
TT_year=2017

In [41]:
# train set
data_train=data_ori[data_ori['Year']<TT_year]

In [42]:
# test set
data_test=data_ori[len(data_train)-window_size+1:]
data_test.index=range(len(data_test))

In [43]:
train = data_train[variables+targets]
test = data_test[variables+targets]

In [44]:
feature_cols = variables
label_cols = ['warning']

train_feature = train[feature_cols]
train_label = train[label_cols]

test_feature = test[feature_cols]
test_label = test[label_cols]

In [45]:
# preprocess train set
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

In [46]:
# preprocess test set
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

In [47]:
# split data into train-validation
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

In [48]:
# prepare result save
result_GRU=pd.DataFrame(data_train[['Year','Week','warning']])
predict_GRU=pd.DataFrame(data_test[['Year','Week','warning']])

In [49]:
result_GRU=result_GRU[2:]
result_GRU.index=range(len(result_GRU))

In [50]:
predict_GRU=predict_GRU[2:]
predict_GRU.index=range(len(predict_GRU))

In [53]:
# GRU
num=0
fit_num=1000
for i in range(fit_num):
    model = Sequential()
    model.add(GRU(layer_num, 
                   input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                   activation='relu', 
                   return_sequences=False)
              )
    sgd=optimizers.SGD(lr=0.001)
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', patience=50)
    filename =  'tmp_checkpoint.h5'
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

    history = model.fit(x_train, y_train, 
                        epochs=5000, 
                        batch_size=30,
                        validation_data=(x_valid, y_valid), 
                        callbacks=[early_stop, checkpoint])
    
    aa=pd.DataFrame(model.predict(train_feature),columns={'a'})
    bb=pd.DataFrame(model.predict(test_feature),columns={'a'})
    result_GRU['Simulation_'+ str(i)]=aa['a']
    predict_GRU['Simulation_'+ str(i)]=bb['a']
    num=num+1


Epoch 1/5000


/Users/sieunlee/miniforge3/envs/noro/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2022-02-18 23:17:11.264247: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-18 23:17:11.267018: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-18 23:17:11.538442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 1s 58ms/step - loss: 1.8200 - val_loss: 0.6526

Epoch 00001: val_loss improved from inf to 0.65263, saving model to tmp_checkpoint.h5
Epoch 2/5000
 4/12 [=========>....................] - ETA: 0s - loss: 0.6955

2022-02-18 23:17:12.508805: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 0s 25ms/step - loss: 0.5298 - val_loss: 0.2682

Epoch 00002: val_loss improved from 0.65263 to 0.26815, saving model to tmp_checkpoint.h5
Epoch 3/5000
12/12 [==============================] - 0s 24ms/step - loss: 0.2188 - val_loss: 0.2376

Epoch 00003: val_loss improved from 0.26815 to 0.23763, saving model to tmp_checkpoint.h5
Epoch 4/5000
12/12 [==============================] - 0s 24ms/step - loss: 0.1712 - val_loss: 0.1865

Epoch 00004: val_loss improved from 0.23763 to 0.18649, saving model to tmp_checkpoint.h5
Epoch 5/5000
12/12 [==============================] - 0s 25ms/step - loss: 0.1293 - val_loss: 0.1527

Epoch 00005: val_loss improved from 0.18649 to 0.15268, saving model to tmp_checkpoint.h5
Epoch 6/5000
12/12 [==============================] - 0s 24ms/step - loss: 0.1289 - val_loss: 0.1460

Epoch 00006: val_loss improved from 0.15268 to 0.14603, saving model to tmp_checkpoint.h5
Epoch 7/5000
12/12 [==============================] -

12/12 [==============================] - 0s 24ms/step - loss: 0.1041 - val_loss: 0.1198

Epoch 00102: val_loss did not improve from 0.07859
Epoch 103/5000
12/12 [==============================] - 0s 23ms/step - loss: 0.0559 - val_loss: 0.0754

Epoch 00103: val_loss improved from 0.07859 to 0.07537, saving model to tmp_checkpoint.h5
Epoch 104/5000
12/12 [==============================] - 0s 24ms/step - loss: 0.0504 - val_loss: 0.0818

Epoch 00104: val_loss did not improve from 0.07537
Epoch 105/5000
12/12 [==============================] - 0s 24ms/step - loss: 0.0512 - val_loss: 0.0779

Epoch 00105: val_loss did not improve from 0.07537
Epoch 106/5000
12/12 [==============================] - 0s 24ms/step - loss: 0.0527 - val_loss: 0.0787

Epoch 00106: val_loss did not improve from 0.07537
Epoch 107/5000
12/12 [==============================] - 0s 24ms/step - loss: 0.0494 - val_loss: 0.1481

Epoch 00107: val_loss did not improve from 0.07537
Epoch 108/5000
12/12 [========================

2022-02-18 23:18:09.562296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [55]:
result_GRU

,Year,Week,warning,Simulation_0
0,2009,3,1,0.991255
1,2009,4,1,1.034391
2,2009,5,1,1.000608
3,2009,6,1,0.966943
4,2009,7,1,1.121620
...,...,...,...,...
417,2016,49,1,1.195673
418,2016,50,1,1.113575
419,2016,51,1,1.120593
420,2016,52,1,1.196418


In [56]:
predict_GRU

,Year,Week,warning,Simulation_0
0,2017,1,1,1.098290
1,2017,2,1,1.208818
2,2017,3,1,1.246450
3,2017,4,1,1.079130
4,2017,5,1,1.091727
...,...,...,...,...
101,2018,49,1,1.038697
102,2018,50,1,0.885603
103,2018,51,1,0.896249
104,2018,52,1,1.190986


In [57]:
# save the results
result_sum=result_GRU.iloc[:,result_GRU.columns.str.contains("Sim")]
predict_sum=predict_GRU.iloc[:,predict_GRU.columns.str.contains("Sim")]

In [58]:
result_sum.to_excel('result_sum_2007_gru.xlsx',index=None)
predict_sum.to_excel('predict_sum_2007_gru.xlsx',index=None)

In [59]:
result_sum['warning']=train_label
predict_sum['warning']=test_label

/var/folders/0t/ffbj5k35735bbppg5hsklzs40000gn/T/ipykernel_45123/3059916937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_sum['warning']=train_label
/var/folders/0t/ffbj5k35735bbppg5hsklzs40000gn/T/ipykernel_45123/3059916937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_sum['warning']=test_label


In [61]:
# train result
r_tmp=result_GRU.iloc[:,result_GRU.columns.str.contains("Sim")]
result_sum['GRU_mean']=r_tmp.mean(axis=1)
result_sum['GRU_95+']=r_tmp.quantile(axis=1,q=0.975)
result_sum['GRU_95-']=r_tmp.quantile(axis=1,q=0.025)

/var/folders/0t/ffbj5k35735bbppg5hsklzs40000gn/T/ipykernel_45123/531947779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_sum['GRU_mean']=r_tmp.mean(axis=1)


In [62]:
# test result
p_tmp=predict_GRU.iloc[:,predict_GRU.columns.str.contains("Sim")]
predict_sum['GRU_mean']=p_tmp.mean(axis=1)
predict_sum['GRU_95+']=p_tmp.quantile(axis=1,q=0.975)
predict_sum['GRU_95-']=p_tmp.quantile(axis=1,q=0.025)

/var/folders/0t/ffbj5k35735bbppg5hsklzs40000gn/T/ipykernel_45123/159267740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_sum['GRU_mean']=p_tmp.mean(axis=1)


In [70]:
p_v=0.3

In [71]:
for i in result_sum.index:
    if (result_sum.loc[i,'GRU_95+']>=p_v):
        result_sum.loc[i,'prediction_95+']=1
    else:
        result_sum.loc[i,'prediction_95+']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'GRU_95+']>=p_v):
        predict_sum.loc[i,'prediction_95+']=1
    else:
        predict_sum.loc[i,'prediction_95+']=0

In [72]:
for i in result_sum.index:
    if (result_sum.loc[i,'GRU_95-']>=p_v):
        result_sum.loc[i,'prediction_95-']=1
    else:
        result_sum.loc[i,'prediction_95-']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'GRU_95-']>=p_v):
        predict_sum.loc[i,'prediction_95-']=1
    else:
        predict_sum.loc[i,'prediction_95-']=0

In [66]:
for i in result_sum.index:
    if (result_sum.loc[i,'GRU_mean']>=p_v):
        result_sum.loc[i,'prediction']=1
    else:
        result_sum.loc[i,'prediction']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'GRU_mean']>=p_v):
        predict_sum.loc[i,'prediction']=1
    else:
        predict_sum.loc[i,'prediction']=0

Result_ACC_train=pd.DataFrame(columns=['accuracy','Recall','Precision','F1'])
Result_ACC_test=pd.DataFrame(columns=['accuracy','Recall','Precision','F1'])

result_sum.loc[(result_sum['warning']==result_sum['prediction']),'equality']=1
predict_sum.loc[(predict_sum['warning']==predict_sum['prediction']),'equality']=1

result_sum=result_sum.fillna(0)
predict_sum=predict_sum.fillna(0)

Result_ACC_train.loc[0,'accuracy']=result_sum['equality'].mean()
Result_ACC_test.loc[0,'accuracy']=predict_sum['equality'].mean()

Result_ACC_train.loc[0,'Recall']=len(result_sum[(result_sum['warning']==1) & (result_sum['prediction']==1)])/len(result_sum[(result_sum['warning']==1)])
Result_ACC_train.loc[0,'Precision']=len(result_sum[(result_sum['warning']==1) & (result_sum['prediction']==1)])/len(result_sum[(result_sum['prediction']==1)])
Result_ACC_train.loc[0,'F1']=2*Result_ACC_train.loc[0,'Recall']*Result_ACC_train.loc[0,'Precision']/(Result_ACC_train.loc[0,'Recall']+Result_ACC_train.loc[0,'Precision'])

Result_ACC_test.loc[0,'Recall']=len(predict_sum[(predict_sum['warning']==1) & (predict_sum['prediction']==1)])/len(predict_sum[(predict_sum['warning']==1)])
Result_ACC_test.loc[0,'Precision']=len(predict_sum[(predict_sum['warning']==1) & (predict_sum['prediction']==1)])/len(predict_sum[(predict_sum['prediction']==1)])
Result_ACC_test.loc[0,'F1']=2*Result_ACC_test.loc[0,'Recall']*Result_ACC_test.loc[0,'Precision']/(Result_ACC_test.loc[0,'Recall']+Result_ACC_test.loc[0,'Precision'])

display(Result_ACC_train)

display(Result_ACC_test)

,accuracy,Recall,Precision,F1
0,0.995261,0.989637,0.922705,0.955


,accuracy,Recall,Precision,F1
0,0.90566,0.852459,0.881356,0.866667


In [74]:
result_save=result_sum[['warning', 'prediction_95-','prediction','prediction_95+']]
predict_save=predict_sum[['warning', 'prediction_95-','prediction','prediction_95+']]

In [75]:
result_save.to_excel('result_save_gru.xlsx',index=None)
predict_save.to_excel('predict_save_gru.xlsx',index=None)